In [1]:
from pynvml import *
from py3nvml import *
import pandas as pd
import numpy as np
import paramiko
import psutil
import time
from multiprocessing import Pool
import datetime

In [2]:
a = pd.DataFrame()

In [3]:
a.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n  </tbody>\n</table>'

In [16]:
with open('/data/theory/robustopt/krisgrg/pass.txt', 'r') as fp:
    pswd = fp.read().rstrip()

def populate_df(ssh_client, machine): 
    i = int(machine.split('-')[-1])
    print(f'{i};')
    flag_done = False 
    _, stdout, stderr = ssh_client.exec_command('source /data/theory/robustopt/krisgrg/.zshrc \n conda activate rob \n python3 /data/theory/robustopt/krisgrg/adversarial_generalization/cl.py')
    for _ in range(5):
        time.sleep(1)
        if stdout.channel.eof_received:
            stdout.channel.close()
            z = stdout.readline()
            print(z.split(';'))
            free_gpus, mem_used, gpu_util, cpu_util, __ = z.split(';')

            for j in range(deviceCount):
                handle = nvmlDeviceGetHandleByIndex(j)
                mem_info = nvmlDeviceGetMemoryInfo(handle)
                util = nvmlDeviceGetUtilizationRates(handle)

            a.at[i, 'node'] = machine
            a.at[i, 'mem_used'] = str(round(mem_info.used / mem_info.total, 2)) + '%'
            z = [float(util) for (i, util) in enumerate(gpu_util.split(', '))]
            a.at[i, 'GPU utilization (only taken)'] = str(np.mean(z)) + '%'
            a.at[i, 'CPU utilization'] = str(cpu_util) + '%'
            a.at[i, 'Free GPUs'] = free_gpus
            flag_done = True
            break

    if not flag_done:
        a.at[i, ''] = 'UNREACHABLE'
    else:
        a.at[i, ''] = ''
        
if __name__ == '__main__':
    nvmlInit()
    deviceCount = nvmlDeviceGetCount()

    a = pd.DataFrame(columns=['node', 'Free GPUs', 'mem_used', 'GPU utilization (only taken)', 'CPU utilization', ''])

    list_machines = ['deep-gpu-1', 'deep-gpu-2', 'deep-gpu-3', 'deep-gpu-4', 'deep-gpu-5', 'deep-gpu-6', 'deep-gpu-7', 'deep-gpu-8', 'deep-gpu-9', 'deep-gpu-10']
    ssh_clients = []
    for machine in list_machines:
        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(hostname=machine,username='krisgrg', password=pswd)
        ssh_clients.append(ssh_client)

    with Pool(processes=len(list_machines)) as p:
        p.map(populate_df, list_machines)
    html_top = "<html>\
        <head>\
        </head>\
        <meta http-equiv='refresh' content='10' />\
        <body>"
    html_str = html_top + a.to_html(index=False) + '</body>'
    with open('/afs/csail.mit.edu/u/k/krisgrg/public_html/cluster.html', 'w') as fp:
        fp.write(html_str)

1;3;2;4;5;6;7;8;10;9;











AuthenticationException: Authentication failed.

In [29]:
!nvidia-smi --query-gpu=utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv

utilization.gpu [%], utilization.memory [%], memory.total [MiB], memory.free [MiB], memory.used [MiB]
0 %, 0 %, 11178 MiB, 10485 MiB, 693 MiB
0 %, 0 %, 11178 MiB, 11168 MiB, 10 MiB
0 %, 0 %, 11178 MiB, 11168 MiB, 10 MiB
0 %, 0 %, 11178 MiB, 11168 MiB, 10 MiB
0 %, 0 %, 11178 MiB, 11168 MiB, 10 MiB
0 %, 0 %, 11178 MiB, 11168 MiB, 10 MiB
0 %, 0 %, 11178 MiB, 11168 MiB, 10 MiB
0 %, 0 %, 11178 MiB, 11168 MiB, 10 MiB


In [81]:
!nvidia-smi

Thu Jul  2 03:57:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 29%   31C    P8     9W / 250W |    693MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 29%   25C    P8     8W / 250W |     10MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [26]:
!nvidia-smi -h

NVIDIA System Management Interface -- v418.87.01

NVSMI provides monitoring information for Tesla and select Quadro devices.
The data is presented in either a plain text or an XML format, via stdout or a file.
NVSMI also provides several management operations for changing the device state.

Note that the functionality of NVSMI is exposed through the NVML C-based
library. See the NVIDIA developer website for more information about NVML.
Python wrappers to NVML are also available.  The output of NVSMI is
not guaranteed to be backwards compatible; NVML and the bindings are backwards
compatible.

http://developer.nvidia.com/nvidia-management-library-nvml/
http://pypi.python.org/pypi/nvidia-ml-py/
Supported products:
- Full Support
    - All Tesla products, starting with the Kepler architecture
    - All Quadro products, starting with the Kepler architecture
    - All GRID products, starting with the Kepler architecture
    - GeForce Titan products, starting with the Kepler architecture
- L

In [77]:
!nvidia-smi --help-query-compute-apps

List of valid properties to query for the switch "--query-compute-apps=":

Section about Active Compute Processes properties
List of processes having compute context on the device.

"timestamp"
The timestamp of where the query was made in format "YYYY/MM/DD HH:MM:SS.msec".

"gpu_name"
The official product name of the GPU. This is an alphanumeric string. For all products.

"gpu_bus_id"
PCI bus id as "domain:bus:device.function", in hex.

"gpu_serial"
This number matches the serial number physically printed on each board. It is a globally unique immutable alphanumeric value.

"gpu_uuid"
This value is the globally unique immutable alphanumeric identifier of the GPU. It does not correspond to any physical label on the board.

"pid"
Process ID of the compute application

"process_name" or "name"
Process Name

"used_gpu_memory" or "used_memory"
Amount memory used on the device by the context. Not available on Windows when running in WDDM mode because Windows KMD manages all the memory not NV

In [80]:
!nvidia-smi --query-compute-apps=pid,process_name,used_memory,gpu_name --format=csv

pid, process_name, used_gpu_memory [MiB], gpu_name
10760, /opt/miniconda/bin/python, 683 MiB, GeForce GTX 1080 Ti
